In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()

In [3]:
ee.Initialize()

In [4]:
from utils.utils import *
from utils.ee_utils import *
from utils.utils import TextColors as c

In [5]:
df = read_csv("LUCAS_2015_Germany.csv")
df

,Piont_id,long,lat
0,40463116,6.072879,51.088722
1,40463188,6.016994,51.734784
2,40523090,6.177697,50.858239
3,40523108,6.164374,51.019789
4,40543118,6.185448,51.110477
...,...,...,...
1681,46543152,14.786924,51.380586
1682,46563124,14.789346,51.128352
1683,46603108,14.831361,50.982524
1684,46623118,14.869216,51.070995


In [6]:
create_folder_if_not_exists('l8_images')

Folder "l8_images" already exists in the current working directory.


In [7]:
# Config
IMAGE_COLLECTION = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
WINTER_RANGES = [('2015-01-01', '2015-02-28'), ('2015-11-01', '2015-12-31')]
NONE_WINTER_RANGES = (WINTER_RANGES[0][1], WINTER_RANGES[1][0])
#Required Landsat bands and RS indices
BANDS = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','clayIndex','ferrousIndex','carbonateIndex','rockOutcropIndex', 'NDVI']
DFOLDER = 'l8_images//'


In [8]:
winter_ragne_1 = ee.Filter.date(WINTER_RANGES[0][0], WINTER_RANGES[0][1])
winter_range_2 = ee.Filter.date(WINTER_RANGES[1][0], WINTER_RANGES[1][1])
winter_range_filter = ee.Filter.Or(winter_ragne_1, winter_range_2)

non_winter_range_filter = ee.Filter.date(NONE_WINTER_RANGES[0], NONE_WINTER_RANGES[1])

for index, row in df.iterrows():
    print(f'{c.BOLD} Point {int(row["Piont_id"])} {c.ENDC}' , end=" Cheking for Winter Image -> ")
    loop_roi = get_square_roi(row['lat'],row['long'], roi_size=1920, return_gee_object=True)
    
    # FIRST: Checkig if we can find an image in the WINTER_RANGE
    l8 = IMAGE_COLLECTION.filter(winter_range_filter)\
                         .filterBounds(loop_roi).sort('system:time_start')
    if l8.size().getInfo() > 0:                     
        #clip the image collection to the roi                     
        l8 = l8.map(lambda img: img.clip(loop_roi))
        
        # discard images with high null pixels
        l8 = l8.map(lambda img: img.set('not_null_pixels', get_not_nulls_ratio(img,loop_roi)))
        l8 = l8.filter(ee.Filter.gt('not_null_pixels',0.7))
        
        # Ratiometric correction
        l8 = l8.map(lambda img: radiometric_correction(img))
        # add roi cloud cover and cloud shadow property # the index 2 is for the combination of cloud and cloud shadow mask
        l8 = l8.map(lambda img: img.set('roi_cloud_cover', get_mask_ones_ratio(get_cloud_mask(img)[2])))
        l8 = l8.filter(ee.Filter.lt('roi_cloud_cover',10)) 
        
        # add roi snow cover property
        l8 = l8.map(lambda img: img.set('roi_snow_cover', get_mask_ones_ratio(get_snow_mask(img))))
        l8 = l8.filter(ee.Filter.lt('roi_snow_cover',10)) 

    if l8.size().getInfo() > 0:
        print (f'{c.OKGREEN} Found {c.ENDC}')
        l8_image = add_mineral_indices(l8.sort('system:time_start').first())

    else: # if we can't find an image in the WINTER_RANGE, we check the NONE_WINTER_RANGE
        print (f'{c.FAIL} Not Found {c.ENDC}', end=" Cheking for None Winter Image -> ")
        l8 = IMAGE_COLLECTION.filter(non_winter_range_filter)\
                             .filterBounds(loop_roi).sort('system:time_start')
                             
        #clip the image collection to the roi                     
        l8 = l8.map(lambda img: img.clip(loop_roi))
                    
        # discard images with high null pixels
        l8 = l8.map(lambda img: img.set('not_null_pixels', get_not_nulls_ratio(img,loop_roi)))
        l8 = l8.filter(ee.Filter.gt('not_null_pixels',0.7))

        # Ratiometric correction
        l8 = l8.map(lambda img: radiometric_correction(img))
        
        # add roi cloud cover and cloud shadow property # the index 2 is for the combination of cloud and cloud shadow mask
        l8 = l8.map(lambda img: img.set('roi_cloud_cover', get_mask_ones_ratio(get_cloud_mask(img)[2])))
        l8 = l8.filter(ee.Filter.lt('roi_cloud_cover',10)) 
        
        # add roi snow cover property
        l8 = l8.map(lambda img: img.set('roi_snow_cover', get_mask_ones_ratio(get_snow_mask(img))))
        l8 = l8.filter(ee.Filter.lt('roi_snow_cover',10)) 
        
        l8 = l8.map(lambda img: img.set('roi_mean_ndvi', get_mean_ndvi(img)))  
        
        if l8.size().getInfo() > 0:
            print(f'{c.OKGREEN} Found {c.ENDC}')
            l8_image = add_mineral_indices(l8.sort('roi_mean_ndvi').first())
        else:
            print(f'{c.FAIL} No Image Found For this ROI! {c.ENDC}')

    if l8.size().getInfo() > 0:    
        date = ee.Date(l8_image.get('system:time_start')).format('YYMMdd').getInfo()
        name = f"{int(row['Piont_id'])}_{date}.tif"
        geemap.download_ee_image(l8_image.select(BANDS), DFOLDER+ name,crs='EPSG:3857', scale=30,region = loop_roi)
                       
    #print('Date:',milsec2date(l8.aggregate_array('system:time_start').getInfo()))
    
    if index == 100:
        break

 Point 40463116  Cheking for Winter Image ->  Found 


40463116_150107.tif: |          | 0.00/1.02M (raw) [  0.0%] in 00:00 (eta:     ?)

 Point 40463188  Cheking for Winter Image ->  Found 


40463188_151123.tif: |          | 0.00/1.04M (raw) [  0.0%] in 00:00 (eta:     ?)

 Point 40523090  Cheking for Winter Image ->  Not Found  Cheking for None Winter Image ->  Found 


...

40603138_150107.tif: |          | 0.00/1.02M (raw) [  0.0%] in 00:00 (eta:     ?)

 Point 40623160  Cheking for Winter Image ->  Found 


40623160_150107.tif: |          | 0.00/1.03M (raw) [  0.0%] in 00:00 (eta:     ?)

 Point 40642990  Cheking for Winter Image ->  Not Found  Cheking for None Winter Image ->  Found 


40642990_150711.tif: |          | 0.00/970k (raw) [  0.0%] in 00:00 (eta:     ?)

## Testing some outputs

In [9]:
date = ee.Date(l8_image.get('system:time_start')).format('YYMMdd').getInfo()
date

'150711'

In [10]:
Map = geemap.Map()
# l8_c_img = l8.sort('roi_mean_ndvi',True).first()
l8_c_img = l8.sort('system:time_start',True).first()
print(l8_c_img.get('roi_mean_ndvi').getInfo())
visualization = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0.0,
  'max': 0.3,
}

cloud = get_cloud_mask(l8_c_img)[2]


Map.addLayer(cloud, {'min': 0, 'max': 1, 'palette': ['black','red']}, 'cloud')
Map.addLayer(l8_c_img, visualization, 'rgb')
Map.centerObject(loop_roi)
Map

0.6243017765281128


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…